In [1]:
import os
import gc
import time
import threading
from datetime import datetime
import urllib3
from itertools import permutations
from dataclasses import dataclass
import pandas as pd
import numpy as np
from numpy import vectorize
from numpy.random import SeedSequence
from typing import Optional

In [2]:
EXECUTION_NAME = datetime.now().replace(second=0, microsecond=0).isoformat()
PREPROCESSING_EXECUTION_NAME = "2024-05-12T20:24:00"
OUTPUT_PATH = '../output/'

EXECUTION_DURATION_S = 14*24*60*60; NS_TO_S = 1000000000

with open(OUTPUT_PATH+PREPROCESSING_EXECUTION_NAME+'-SEED.csv', 'r') as f:
    seedz = np.vectorize(int)(f.read().split(','))
assert(seedz is not None)    

RANDOM_GEN = np.random.default_rng(seed=SeedSequence(entropy=list(map(abs,seedz))))

In [3]:
DF = pd.read_csv(OUTPUT_PATH+PREPROCESSING_EXECUTION_NAME+'-EVENTS.csv.gz',
                 index_col='id')

In [4]:
_count = -1
_app_id = -1
def getCount(app_id):
    global _count,_app_id
    if app_id == _app_id:
        _count += 1
    else:
        _count = 0
        _app_id = app_id
    return _count

In [6]:
STATE = {
    "global":{'current_requests': set(),
        'arrival_dt': [],
        'departure_dt': [],
        'arrival_depature_dt': [],
        'departure_arrival_dt': [],
        'last_arrival': None,
        'last_departure': None
    },
    "app":dict(
        [*[app_id,
            {'current_requests': set(),
             'arrival_dt': [],
             'departure_dt': [],
             'arrival_depature_dt': [],
             'departure_arrival_dt': [],
             'last_arrival': None,
             'last_departure': None,
             'func_map': {func: getCount(app_id) \
                           for func in DF[DF["app_id"] == app_id]['func_id'].unique()}}]] 
        for app_id in DF["app_id"].unique()),
    "func":dict(
        [*[func_id,
            {'current_requests': set(),
             'request_count': 0
             'arrival_dt': [],
             'departure_dt': [],
             'arrival_depature_dt': [],
             'departure_arrival_dt': [],
             'last_arrival': None,
             'last_departure': None}]] 
        for func_id in DF["func_id"].unique()),
}

In [8]:
DF.columns

Index(['ts', 'event_type', 'req_id', 'func_id', 'app_id', 'beg_ns', 'end_ns',
       'dur_ns'],
      dtype='object')

In [25]:
@dataclass
class EventMod:
    ts: int
    level: int
    ID: int
    request_count: Optional[int] = None
    arrival_dt: Optional[int] = None
    departure_dt: Optional[int] = None
    arrival_depature_dt: Optional[int] = None
    departure_arrival_dt: Optional[int] = None

In [26]:
events: list[EventMod] = []

In [ ]:
for event in DF.itertuples():
    func = STATE["func"][event.func_id]
    eventMod = EventMod(event.ts, 0, event.func_id)
    if func['last_departure'] is not None:
        if event.event_type == 1:
            eventMod.request_count = 
            event.ts - func['last_arrival'].ts
        else:
            pass
    elif func['last_arrival'] is not None:
        if event.event_type == 1:
            pass
        else:
            pass
    else:
        if event.event_type == 1:
            pass
        else:
            pass
    app = STATE["app"][event.app_id]
    if app['last_departure'] is not None:
        if event.event_type == 1:
            pass
        else:
            pass
    elif app['last_arrival'] is not None:
        if event.event_type == 1:
            pass
        else:
            pass
    else:
        if event.event_type == 1:
            pass
        else:
            pass
    glb = STATE["global"]
    if glb['last_departure'] is not None:
        if event.event_type == 1:
            pass
        else:
            pass
    elif glb['last_arrival'] is not None:
        if event.event_type == 1:
            pass
        else:
            pass
    else:
        if event.event_type == 1:
            pass
        else:
            pass
        

            
        

In [ ]:
DF['group'] = (DF['ts'] / NS_TO_S + 1).astype(int)

In [49]:
@dataclass
class Bucket:
    level = 0
    timestamp: int 
    functionID: int
    applicationID: int
    count: int
    lastArrivalDt: float
    lastDepartureDt: float
    avgCount: Optional[float] = 0
    maxCount: Optional[int] = 0   
    arrivalCount: Optional[int] = 0
    departureCount: Optional[int] = 0
    avgRunningAge: Optional[float] = 0
    virtualColdStarts: Optional[int] = 0
    idlePc: Optional[float] = 0

@dataclass
class AppBucket:
    level = 1
    timestamp: int 
    applicationID: int
    count: int
    lastArrivalDt: float
    lastDepartureDt: float
    avgCount: Optional[float] = 0
    maxCount: Optional[int] = 0   
    arrivalCount: Optional[int] = 0
    departureCount: Optional[int] = 0
    avgRunningAge: Optional[float] = 0
    virtualColdStarts: Optional[int] = 0
    idlePc: Optional[float] = 0

@dataclass
class GlobalBucket:
    level = 2
    timestamp: int 
    count: int
    lastArrivalDt: float
    lastDepartureDt: float
    avgCount: Optional[float] = 0
    maxCount: Optional[int] = 0   
    arrivalCount: Optional[int] = 0
    departureCount: Optional[int] = 0
    avgRunningAge: Optional[float] = 0
    virtualColdStarts: Optional[int] = 0
    idlePc: Optional[float] = 0

In [43]:
DF.columns

Index(['ts', 'event_type', 'req_id', 'func_id', 'app_id', 'beg_ns', 'end_ns',
       'dur_ns', 'group'],
      dtype='object')

In [8]:
buckets = [dict(),]*(EXECUTION_DURATION_S+1)
buckets[0]['global'] = GlobalBucket(0, 0, 0, 0)
for grp in range(1, len(buckets)):
    df_grp = DF[DF['group'] == grp]
    bucket = buckets[grp]
    
    for event in df_grp.itertuples():
        
        if STATE['func'][event.func_id]['count'] == None:
            pass #init func
            if STATE['app'][event.app_id]['count'] == None:
                pass #init app
                if STATE['global']['count'] == None:
                   STATE['global']['current_requests'].add(event.req)
                   STATE['global']['count'] = 0 
            
        if event.func_id not in bucket:

            func_state = STATE['func'][event.func_id]
            bucket[event.func_id] = Bucket(event.ts, event.func_id, event.app_id,
                                           
                                        
        else:
            pass # departure
        
        
        
        
        
        
    

In [45]:
DF.iloc[0]['event_type']

1